In [1]:
# Pastikan PyTorch telah terinstal di Google Colab
!pip install torch torchvision

# Import library yang dibutuhkan
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Unggah file 'wine.data' terlebih dahulu di Colab

# Membaca dataset
wine_data = pd.read_csv('wine.data', header=None)

# Define features and target
X = wine_data.iloc[:, 1:]  # Semua kolom kecuali yang pertama (fitur)
y = wine_data.iloc[:, 0]   # Kolom pertama sebagai target

# Split data ke dalam data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize fitur menggunakan StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Konversi data ke PyTorch tensor
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Membuat DataLoader
batch_size = 32  # Placeholder untuk eksperimen
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
# Definisi Model Vanilla MLP
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, activation_fn):
        super(MLPModel, self).__init__()
        layers = []
        current_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(activation_fn)
            current_size = hidden_size
        layers.append(nn.Linear(current_size, 1))  # Output layer
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Placeholder untuk model
input_size = X_train_tensor.shape[1]
hidden_sizes = [32]  # Jumlah neuron per hidden layer
activation_fn = nn.ReLU()  # Aktivasi yang dipilih
model = MLPModel(input_size, hidden_sizes, activation_fn)

# Definisi Loss dan Optimizer
learning_rate = 0.01  # Placeholder untuk eksperimen
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [4]:
# Fungsi untuk melatih model
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs):
    train_losses = []
    test_losses = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))

        # Evaluasi pada data uji
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                test_loss += loss.item()
        test_losses.append(test_loss / len(test_loader))

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}")
    return train_losses, test_losses

# Melatih model dengan hyperparameter awal
num_epochs = 50  # Placeholder untuk eksperimen
train_losses, test_losses = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs)


Epoch 1/50, Train Loss: 4.9909, Test Loss: 2.5333
Epoch 2/50, Train Loss: 2.1518, Test Loss: 0.9112
Epoch 3/50, Train Loss: 0.5812, Test Loss: 0.1550
Epoch 4/50, Train Loss: 0.4126, Test Loss: 0.1931
Epoch 5/50, Train Loss: 0.4001, Test Loss: 0.2614
Epoch 6/50, Train Loss: 0.2522, Test Loss: 0.1879
Epoch 7/50, Train Loss: 0.1564, Test Loss: 0.1624
Epoch 8/50, Train Loss: 0.1283, Test Loss: 0.1514
Epoch 9/50, Train Loss: 0.1052, Test Loss: 0.1287
Epoch 10/50, Train Loss: 0.0918, Test Loss: 0.1022
Epoch 11/50, Train Loss: 0.0746, Test Loss: 0.0878
Epoch 12/50, Train Loss: 0.0748, Test Loss: 0.0939
Epoch 13/50, Train Loss: 0.0621, Test Loss: 0.0908
Epoch 14/50, Train Loss: 0.0612, Test Loss: 0.0831
Epoch 15/50, Train Loss: 0.0539, Test Loss: 0.0796
Epoch 16/50, Train Loss: 0.0507, Test Loss: 0.0803
Epoch 17/50, Train Loss: 0.0490, Test Loss: 0.0831
Epoch 18/50, Train Loss: 0.0469, Test Loss: 0.0812
Epoch 19/50, Train Loss: 0.0458, Test Loss: 0.0767
Epoch 20/50, Train Loss: 0.0466, Test Lo

In [5]:
# Eksperimen dengan berbagai konfigurasi
hidden_layers = [[4], [8], [16, 16], [32, 32], [64, 64]]
activation_functions = [nn.ReLU(), nn.Sigmoid(), nn.Tanh()]
learning_rates = [1.0, 0.1, 0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64, 128]

results = []

for hidden_size in hidden_layers:
    for activation in activation_functions:
        for lr in learning_rates:
            for batch in batch_sizes:
                # Update DataLoader
                train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
                test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=False)

                # Initialize model, criterion, and optimizer
                model = MLPModel(input_size, hidden_size, activation)
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Train model
                train_losses, test_losses = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10)

                # Save results
                results.append({
                    'hidden_layers': hidden_size,
                    'activation': activation.__class__.__name__,
                    'learning_rate': lr,
                    'batch_size': batch,
                    'final_train_loss': train_losses[-1],
                    'final_test_loss': test_losses[-1]
                })


Epoch 1/10, Train Loss: 12.8019, Test Loss: 4.5460
Epoch 2/10, Train Loss: 4.7337, Test Loss: 2.0705
Epoch 3/10, Train Loss: 2.7208, Test Loss: 0.8853
Epoch 4/10, Train Loss: 1.1474, Test Loss: 1.7560
Epoch 5/10, Train Loss: 0.8223, Test Loss: 0.6513
Epoch 6/10, Train Loss: 0.6080, Test Loss: 0.7649
Epoch 7/10, Train Loss: 0.6281, Test Loss: 0.7121
Epoch 8/10, Train Loss: 0.6026, Test Loss: 0.7099
Epoch 9/10, Train Loss: 0.6039, Test Loss: 0.7276
Epoch 10/10, Train Loss: 0.6142, Test Loss: 0.6716
Epoch 1/10, Train Loss: 38.9227, Test Loss: 20.5700
Epoch 2/10, Train Loss: 8.7310, Test Loss: 2.6362
Epoch 3/10, Train Loss: 3.4417, Test Loss: 3.7604
Epoch 4/10, Train Loss: 3.8292, Test Loss: 2.3901
Epoch 5/10, Train Loss: 2.7686, Test Loss: 0.9373
Epoch 6/10, Train Loss: 0.7639, Test Loss: 0.8306
Epoch 7/10, Train Loss: 1.1695, Test Loss: 0.7964
Epoch 8/10, Train Loss: 0.9611, Test Loss: 0.8116
Epoch 9/10, Train Loss: 0.6990, Test Loss: 1.1504
Epoch 10/10, Train Loss: 0.7368, Test Loss: 0.

In [6]:
# Konversi hasil ke DataFrame untuk analisis
results_df = pd.DataFrame(results)

# Tampilkan kombinasi hyperparameter terbaik
best_result = results_df.loc[results_df['final_test_loss'].idxmin()]
print("Best Hyperparameter Combination:")
print(best_result)

# Simpan hasil ke file CSV
results_df.to_csv('hyperparameter_results.csv', index=False)


Best Hyperparameter Combination:
hidden_layers       [16, 16]
activation              Tanh
learning_rate            0.1
batch_size                16
final_train_loss    0.030011
final_test_loss     0.007185
Name: 164, dtype: object
